# T5 英文文本生成
## 教學目標
使用T5模型根據英文關鍵字生成完整的句子

## 適用對象
 - 已對python的基本語法和有一定瞭解和掌握程度
 - 對深度學習的基本概念有初步的認識

## 執行方法
在 Jupyter notebook 中，選取想要執行的區塊後，使用以下其中一種方法執行
 - 上方工具列中，按下 Cell < Run Cells 執行
 - 使用快捷鍵 Shift + Enter 執行

## 大綱
 - [安裝套件](#安裝套件)
 - [載入T5模型](#載入T5模型)
 - [資料處理](#資料處理)
 - [超參數](#超參數)
 - [訓練](#訓練)
 - [驗證](#驗證)


## 安裝套件
 - transformers (4.37.0) huggingface讀取模型的套件
 - datasets (2.16.1) huggingface讀取資料集的套件
 - torcheval (0.0.7) 各種評價標準

In [1]:
! pip install transformers
! pip install datasets
! pip install torcheval
! pip install pytorch-ignite

In [2]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from ignite.metrics import Rouge
import re

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from transformers import BertTokenizer, RobertaTokenizer,BertTokenizerFast


## 載入T5模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)

In [4]:
t5_model = T.T5ForConditionalGeneration.from_pretrained("google-t5/t5-large", cache_dir="./cache/").to(device)
t5_tokenizer = T.T5Tokenizer.from_pretrained("google-t5/t5-large", cache_dir="./cache/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want 

In [5]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [6]:
bert_tokenizer_fast = BertTokenizerFast.from_pretrained("bert-base-chinese",add_special_tokens=False,do_lower_case=True)

In [7]:
gpt2_model = T.GPT2LMHeadModel.from_pretrained("gpt2", cache_dir="./cache/").to(device)
gpt2_tokenizer = T.GPT2Tokenizer.from_pretrained("gpt2", cache_dir="./cache/")
# gpt2_model = T.GPT2ForSequenceClassification.from_pretrained("gpt2", cache_dir="./cache/").to(device)
# gpt2_tokenizer = T.GPT2Tokenizer.from_pretrained("gpt2", cache_dir="./cache/")
gpt2_tokenizer.padding_side = "left"
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
# gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [8]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

In [9]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
bert_model = BertForMaskedLM.from_pretrained("bert-base-chinese").to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

## 資料處理
 - 使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料
 - 使用“/”將每個輸入的關鍵字和每個輸出鏈接起來

In [11]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    # t5 tokenizer不能處理中文，所以換成bert的試試看
    model_inputs = bert_tokenizer.batch_encode_plus([each["text"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    model_outputs = bert_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    # bert
    # model_inputs = roberta_tokenizer.batch_encode_plus([each["text"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    # model_outputs = roberta_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    # bertfast
    # model_inputs = bert_tokenizer.batch_encode_plus([each["text"] for each in sample], padding=True, truncation=True, return_tensors="pt",return_offsets_mapping=True,add_special_tokens=False)
    # model_outputs = bert_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt",return_offsets_mapping=True,add_special_tokens=False)
    # bert model
    # model_inputs = bert_tokenizer([each["text"] for each in sample], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    # model_outputs = bert_tokenizer([each["summary"] for each in sample], padding='max_length', truncation=True, max_length=128, return_tensors="pt")

    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)

class CommonGenDataset(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation", "test"]
        # data_df = load_dataset("allenai/common_gen", split=split, cache_dir="./cache/").to_pandas().groupby("concept_set_idx")

        data_df = load_dataset("hugcyp/LCSTS", split=split, cache_dir="./cache/").to_pandas()
        self.data = []
        # for each in data_df:
        #     targets = "/ ".join([s+"." if not s.endswith(".") else s for s in each[1].target.to_list()])
        #     concepts = ", ".join(each[1].concepts.to_list()[0])
        #     self.data.append({"concepts": concepts, "targets": targets})
        for num in range(0,len(data_df)):
          self.data.append({"summary":data_df['summary'][num],"text":data_df['text'][num]})
        # print(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)



In [12]:
def get_tensor_gpt2(sample):
    input_ids = gpt2_tokenizer.batch_encode_plus([each["text"] for each in sample], padding=True, truncation=True, return_tensors="pt")["input_ids"].to(device)
    labels = gpt2_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt")["input_ids"].to(device)
    return input_ids, labels
class CommonGenDataset_gpt2(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation", "test"]
        data_df = load_dataset("hugcyp/LCSTS", split=split, cache_dir="./cache/").to_pandas()
        self.data = [{"summary":data_df['summary'][num],"text":data_df['text'][num]} for num in range(len(data_df))]

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

## 超參數
 - 學習率 (learning rate): 1e-5
 - 訓練輪數 (epochs): 3
 - 優化器 (optimizer): AdamW
 - 批次大小 (batch size): 8
 - 評量指標 (evaluation matrics)Rouge-2

In [13]:
lr = 1e-5
epochs = 1
optimizer = AdamW(t5_model.parameters(), lr = 1e-5)
# optimizer = AdamW(bert_model.parameters(), lr = 1e-5)
train_batch_size = 8
validation_batch_size = 8
common_gen_train = DataLoader(CommonGenDataset(split="train").data[:200], collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
common_gen_validation = DataLoader(CommonGenDataset(split="validation").data[:200], collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

In [14]:
lr = 1e-5
epochs = 1
optimizer = AdamW(gpt2_model.parameters(), lr=lr)
train_batch_size = 8
validation_batch_size = 8
common_gen_train_gpt2 = DataLoader(CommonGenDataset_gpt2(split="train").data[:200], collate_fn=get_tensor_gpt2, batch_size=train_batch_size, shuffle=True)
common_gen_validation_gpt2 = DataLoader(CommonGenDataset_gpt2(split="validation").data[:200], collate_fn=get_tensor_gpt2, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [15]:
def evaluate(model):
    pbar = tqdm(common_gen_validation)
    pbar.set_description(f"Evaluating")

    for inputs, targets in pbar:
        # output = [re.split(r"[/]", each.replace("<pad>", "")) for each in t5_tokenizer.batch_decode(model.generate(inputs, max_length=50))]
        # targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in t5_tokenizer.batch_decode(targets)]
        output = [re.split(r"[/]", each.replace("<pad>", "")) for each in bert_tokenizer.batch_decode(model.generate(inputs, max_length=50))]
        targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in bert_tokenizer.batch_decode(targets)]
        # output = [re.split(r"[/]", each.replace("<pad>", "")) for each in roberta_tokenizer.batch_decode(model.generate(inputs, max_length=50))]
        # targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in roberta_tokenizer.batch_decode(targets)]
        for i in range(len(output)):
            sentences = [s.replace('.', ' .').split() for s in output[i]]
            ground_thruths = [t.replace('.', ' .').split() for t in targets[i]]
            for s in sentences:
                rouge.update(([s], [ground_thruths]))
    return rouge.compute()


In [16]:
!pip install rouge_score
from rouge_score import rouge_scorer

In [17]:
# def evaluate_gpt2(model):
#     pbar = tqdm(common_gen_validation_gpt2)
#     pbar.set_description(f"Evaluating")

#     for inputs, targets in pbar:

#         output = gpt2_model.generate(inputs, max_length=50, num_return_sequences=1, temperature=1.0,pad_token_id=gpt2_model.config.eos_token_id)
#         output = [gpt2_tokenizer.decode(ids, skip_special_tokens=True) for ids in output]
#         targets = [gpt2_tokenizer.decode(ids, skip_special_tokens=True) for ids in targets]
#         for i in range(len(output)):
#             rouge.update([(output[i].split(), targets[i].split())])
#     return rouge.compute()

import torch

def evaluate_gpt2(model, tokenizer, data_loader):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
    total_rouge2 = 0
    num_samples = 0

    with torch.no_grad():
        for inputs, targets in data_loader:
            # 计算需要填充的数量
            padding_needed = inputs.size(1) - targets.size(1)

            # 创建填充参数
            padding = (0, padding_needed)

            # 在目标序列后面填充
            padded_targets = torch.nn.functional.pad(targets, padding, value=tokenizer.pad_token_id)

            # 创建注意力遮罩
            attention_mask = inputs.ne(tokenizer.pad_token_id)

            # 将填充标记 ID 替换为 EOS（结束）标记 ID
            inputs[inputs == tokenizer.pad_token_id] = tokenizer.eos_token_id

            # 生成摘要
            generated_summary = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=50)
            generated_summary = [tokenizer.decode(summary, skip_special_tokens=True) for summary in generated_summary]

            # 计算 ROUGE-2 分数
            for generated, target in zip(generated_summary, targets):
                scores = scorer.score(generated, tokenizer.decode(target, skip_special_tokens=True))
                total_rouge2 += scores['rouge2'].fmeasure
                num_samples += 1

    return total_rouge2 / num_samples

# 使用示例



## 訓練
 - 將資料成批次輸入T5模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = t5_model(input_ids=inputs, labels=targets).loss
        loss.backward()

        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    # torch.save(t5_model, f'./saved_models/ep{ep}.mod')
    print(f"Rouge-2 score on epoch {ep}:", evaluate(t5_model))

Training epoch [1/1]:   8%|▊         | 2/25 [01:41<18:48, 49.06s/it, loss=12.1]

In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train_gpt2)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        # print()
        # print(inputs)
        # print(targets)
        padding_needed = inputs.size(1) - targets.size(1)

        # 定义填充参数
        padding = (0, padding_needed)
        # 在目标序列后面填充
        padded_targets = torch.nn.functional.pad(targets, padding, value=gpt2_tokenizer.pad_token_id)

        # 创建注意力遮罩
        attention_mask = inputs.ne(gpt2_tokenizer.pad_token_id)

        # 将填充标记 ID 替换为 EOS（结束）标记 ID
        inputs[inputs == gpt2_tokenizer.pad_token_id] = gpt2_tokenizer.eos_token_id

        # # 计算损失函数
        # loss = gpt2_model(input_ids=inputs, labels=padded_targets, attention_mask=attention_mask).loss
        # # 在张量后面填充
        # padded_targets = torch.nn.functional.pad(targets, padding, value=0)
        # pad_token_id=gpt2_model.config.eos_token_id
        # # pad_token_id = gpt2_tokenizer.pad_token_id
        # inputs[inputs == pad_token_id] = gpt2_tokenizer.eos_token_id  # 將填充標記 ID 替換為 EOS（結束）標記 ID

        # # 創建注意力遮罩
        # attention_mask = inputs.ne(gpt2_tokenizer.pad_token_id)
        loss = gpt2_model(input_ids=inputs, labels=padded_targets,attention_mask=attention_mask).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item())
    # print(f"Rouge-2 score on epoch {ep}:", evaluate_gpt2(gpt2_model))
    rouge_score = evaluate_gpt2(gpt2_model, gpt2_tokenizer, common_gen_validation_gpt2)
    print(f"Average ROUGE-2 score: {rouge_score}")